In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler 
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

In [ ]:
# Put the directory of the datasets here
os.chdir(r"/content/drive/My Drive/STA-ML")

path = os.getcwd()
print(path)

/content/drive/My Drive/STA-ML


In [ ]:
# Load data into DataFrame
train_data = pd.read_csv(path+'/dexter/dexter_train.csv',header=None)
test_data = pd.read_csv(path+'/dexter/dexter_valid.csv',header=None)
train_labels = pd.read_csv(path+'/dexter/dexter_train.labels',header=None)
test_labels = pd.read_csv(path+'/dexter/dexter_valid.labels',header=None)
# Add a foward column  of 1's on training datafor the linear constant

#data.insert(0,'const',1)

print(train_data)
print(train_labels)

     0      1      2      3      4      ...  19995  19996  19997  19998  19999
0        0      0      0      0      0  ...      0      0      0      0     56
1        0      0      0      0      0  ...      0      0      0      0      0
2        0      0      0      0      0  ...      0      0      0      0      0
3        0      0      0      0      0  ...      0      0      0      0      0
4        0      0      0      0      0  ...      0      0      0      0      0
..     ...    ...    ...    ...    ...  ...    ...    ...    ...    ...    ...
295      0      0      0      0      0  ...      0      0      0      0      0
296      0      0      0      0      0  ...      0      0      0      0      0
297      0      0      0      0      0  ...      0      0      0      0      0
298      0      0      0      0      0  ...      0      0      0      0      0
299      0      0      0      0      0  ...      0      0      0      0      0

[300 rows x 20000 columns]
     0
0    1
1   -1
2  

In [ ]:
ss=StandardScaler()
ss.fit(train_data)
X_train=ss.transform(train_data)
X_test=ss.transform(test_data)

print(X_train.shape)

(300, 20000)


In [ ]:
tf_X_train = X_train.T
tf_X_test = X_test.T
y_train=train_labels.replace(-1,0)
y_test=test_labels.replace(-1,0)

In [ ]:
#theta = np.zeros((X_train.shape[1],1)) * 0.00001
hx = np.zeros((1,X_train.shape[0]))

p = 1/(1+np.exp(hx.T))
w = p*(1-p)
print(p.shape, w.shape)
z = (y_train.values - p) / p*(1-p)
Xbar = np.dot(w.T,X_train) / np.sum(w)
Zbar = np.dot(w.T,z) / np.sum(w)

print(z.shape,Xbar.shape, Zbar.shape, w.shape)

(300, 1) (300, 1)
(300, 1) (1, 20000) (1, 1) (300, 1)


In [ ]:
ydiff = y_train.values - Zbar
xdiff = X_train - Xbar

print(ydiff.shape, xdiff.shape)

(300, 1) (300, 20000)


In [ ]:
b1 = np.dot(w,np.dot(ydiff.T,xdiff)) / (np.dot(w.T, xdiff**2) + 0.0001)

b1.shape

(300, 20000)

In [ ]:
np.dot(ydiff.T,xdiff).shape

(1, 20000)

In [ ]:
b0 = Zbar - np.dot(b1,Xbar.T)

b0.shape

(300, 1)

In [ ]:
print(np.reshape(b1[0],(1,-1)).shape)

(1, 20000)


In [ ]:
bj = np.reshape(b1[0],(-1,1))
hj = (np.dot(bj.T,tf_X_train) + b0[0]).T


print(hj) 

#w_new[i] = w_old[i] +b1[index]

[[  8.21727397]
 [-15.67225658]
 [  9.42058112]
 [-16.98749348]
 [ 17.02871632]
 [-15.89469611]
 [ 12.67450437]
 [-13.93497838]
 [ 19.60389049]
 [ 25.08279791]
 [ 16.00053219]
 [ 14.45181007]
 [-17.89696134]
 [ 12.65966698]
 [ 16.43564398]
 [ 16.03944955]
 [-16.07654307]
 [ 23.86449227]
 [-10.87357782]
 [-10.47515373]
 [ 19.16452666]
 [-14.89858502]
 [ 13.28988953]
 [ 14.41631023]
 [ 17.67092691]
 [ 22.16223057]
 [ 17.187478  ]
 [ -8.19963972]
 [-15.18360379]
 [-14.16206853]
 [ 10.7723769 ]
 [-17.4103156 ]
 [-11.83632158]
 [ 19.59303221]
 [ 16.28036224]
 [ 15.22748155]
 [ 12.43041122]
 [-16.89170044]
 [ 15.23394778]
 [-13.17449194]
 [-17.68640853]
 [ -6.07586531]
 [-12.94795134]
 [ 20.34473632]
 [ -7.41659355]
 [-13.71976855]
 [-10.62217881]
 [-22.51121258]
 [-10.18984756]
 [ 18.37036971]
 [-16.17579391]
 [-12.75397197]
 [ 20.02542794]
 [-23.23968168]
 [-11.89241725]
 [-18.57242744]
 [ 21.15336559]
 [ 16.97457727]
 [ 15.71102212]
 [ 15.18048966]
 [ 26.42202048]
 [-14.06570943]
 [-12.70

In [ ]:
weak_loss = log_loss(y_train.values,hj)

weak_loss

9.992007221626415e-16

In [ ]:
cost = []
hj = []
b1_store = []
b0_store = []
print(hx)

for i in range(b1.shape[0]):
  bj = np.reshape(b1[i],(-1,1))
  temp = (np.dot(bj.T,tf_X_train) + b0[i]).T
  hj.append(temp)
  
  weak_loss = log_loss(y_train.values,temp)
  cost.append(weak_loss)

index = np.argmin(cost)

hx = hx + hj[index]

print(hx)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[  8.21727397   8.21727397   8.21727397 ...   8.21727397   8.21727397
 

In [ ]:
print(cost)
print(index)

[9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.992007221626415e-16, 9.99200722

In [ ]:
b1[index].shape

(20000,)

In [ ]:
hx = np.zeros(X_train.shape[0])
loss = []
b1_store = []
b0_store = []

for it in range(10) :
  hj = []
  cost = []
  p = 1/(1+np.exp(hx))
  w = p*(1-p)
  z = np.divide(np.asarray(y_train.values-p), w, out=np.zeros(X_train.shape[0]), where=(w!=0))
  Xbar = np.dot(w.T,X_train) / np.sum(w)
  Zbar = np.dot(w.T,z) / np.sum(w)

  zdiff = z - Zbar
  xdiff = X_train - Xbar
  b1 = np.dot(w,np.dot(zdiff.T,xdiff)) / (np.dot(w.T, xdiff**2) + 0.0001)
  b0 = Zbar - np.dot(b1,Xbar.T)

  for i in range(b1.shape[0]):
    bj = np.reshape(b1[i],(-1,1))
    temp = (np.dot(bj.T,tf_X_train) + b0[i]).T
    hj.append(temp)
    
    weak_loss = log_loss(y_train.values,temp)
    cost.append(weak_loss)
  index = np.argmin(cost)
  loss.append(cost)
  hx = hx + hj[index]
"""
hx.shape

p = 1/(1+np.exp(hx.T))

print(p.shape)
"""

ValueError: ignored

In [ ]:
"""
hx = np.zeros((1,X_train.shape[0]))
loss = []
b1_store = []
b0_store = []

for it in range(10) :
  cost = []
  hj = []
  p = 1/(1+np.exp(hx.T))
  w = p*(1-p)
  if w == 0 :
    z = 
  z = np.divide((y_train.values-p), w, out=np.zeros(X_train.shape[0]), where=(w!=0))
  Xbar = np.dot(w.T,X_train) / np.sum(w)
  Zbar = np.dot(w.T,z) / np.sum(w)

  zdiff = z - Zbar
  xdiff = X_train - Xbar
  b1 = np.dot(w,np.dot(zdiff.T,xdiff)) / (np.dot(w.T, xdiff**2) + 0.0001)
  b0 = Zbar - np.dot(b1,Xbar.T)

  for i in range(b1.shape[0]):
    bj = np.reshape(b1[i],(-1,1))
    temp = (np.dot(bj.T,tf_X_train) + b0[i]).T
    hj.append(temp)
    weak_loss = log_loss(y_train.values,temp)
    cost.append(weak_loss)
  index = np.argmin(cost)
  loss.append(cost)
  b1_store.append(b1)
  b0_store.append(b0)
  hx = hx + hj[index]


In [ ]:
w.shape

In [ ]:
b1.shape

In [ ]:
X_train.shape